In [1]:
import sys
# sys.path.insert(0, '/home/wdwatson2/projects/CAT-Transformer/model')
# sys.path.insert(0, r'C:\Users\smbm2\projects\CAT-Transformer\model')
sys.path.insert(0, '/home/cscadmin/CyberResearch/CAT-Transformer/model')
from testingModel import CATTransformer, MyFTTransformer, Combined_Dataset, train, test, EarlyStopping
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score, recall_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import optuna
from optuna.trial import TrialState

device_in_use = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device_in_use)

cuda


/home/cscadmin/miniconda3/envs/torch-env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# df_train = pd.read_csv('/home/wdwatson2/projects/CAT-Transformer/datasets/higgs/train.csv')
# df_test = pd.read_csv('/home/wdwatson2/projects/CAT-Transformer/datasets/higgs/test.csv')
# df_val = pd.read_csv('/home/wdwatson2/projects/CAT-Transformer/datasets/higgs/validation.csv') 

df_train = pd.read_csv('/home/cscadmin/CyberResearch/CAT-Transformer/datasets/higgs/train.csv')
df_test = pd.read_csv('/home/cscadmin/CyberResearch/CAT-Transformer/datasets/higgs/test.csv')
df_val = pd.read_csv('/home/cscadmin/CyberResearch/CAT-Transformer/datasets/higgs/validation.csv') 

cont_columns = ['lepton_pT', 'lepton_eta', 'lepton_phi',
       'missing_energy_magnitude', 'missing_energy_phi', 'jet1pt', 'jet1eta',
       'jet1phi', 'jet1b-tag', 'jet2pt', 'jet2eta', 'jet2phi', 'jet2b-tag',
       'jet3pt', 'jet3eta', 'jet3phi', 'jet3b-tag', 'jet4pt', 'jet4eta',
       'jet4phi', 'jet4b-tag', 'm_jj', 'm_jjj', 'm_lv', 'm_jlv', 'm_bb',
       'm_wbb', 'm_wwbb']
target = ['class']
cat_columns = []


#CHECKING TO MAKE SURE YOUR LIST IS CORRECT (NO NEED TO TOUCH)
yourlist = cont_columns + cat_columns+target
yourlist.sort()
oglist = list(df_train.columns)
oglist.sort()

assert(yourlist == oglist), "You may of spelled feature name wrong or you forgot to put on of them in the list"

cat_features = ()

target_classes = [max(len(df_train[target].value_counts()), len(df_val[target].value_counts()),len(df_test[target].value_counts()))]
print(target_classes)
# Create a StandardScaler and fit it to the cont features
scaler = StandardScaler()
scaler.fit(df_train[cont_columns])

# Transform the training, test, and validation datasets
df_train[cont_columns] = scaler.transform(df_train[cont_columns])
df_test[cont_columns] = scaler.transform(df_test[cont_columns])
df_val[cont_columns] = scaler.transform(df_val[cont_columns])

#Wrapping in Dataset
train_dataset = Combined_Dataset(df_train, cat_columns=cat_columns, num_columns=cont_columns, task1_column=target[0])
val_dataset = Combined_Dataset(df_val, cat_columns=cat_columns, num_columns=cont_columns, task1_column=target[0])
test_dataset = Combined_Dataset(df_test, cat_columns=cat_columns, num_columns=cont_columns, task1_column=target[0])

batch_size = 256

# Wrapping with DataLoader for easy batch extraction
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

[2]


In [3]:
def objective(trial):
    trial_number = trial.number

    # Define hyperparameters to search over
    alpha = trial.suggest_float('sigma', 0.001, 5, log=True)
    num_layers = trial.suggest_int('num_layers', 1, 5)
    # Ensure that embed_size is divisible by num_layers
    embed_size = trial.suggest_categorical("embed_size", [50, 60, 70, 80, 90, 100, 120, 140, 160, 180, 200, 250, 350, 500])
    heads = trial.suggest_categorical("heads", [1, 5, 10])
    forward_expansion = trial.suggest_int('forward_expansion', 1, 8)
    pre_norm_on = trial.suggest_categorical('prenorm_on', [True, False])
    mlp_scale_classification = trial.suggest_int('mlp_scale_classification', 1, 8)
    decoder_dropout = trial.suggest_categorical('decoder_dropout', [0,.1,.2,.5])
    classification_dropout = trial.suggest_categorical('class_drop', [0,.1,.2,.5])

    learning_rate = trial.suggest_float('learning_rate', 0.00001, 0.001, log=True)
    weight_decay = trial.suggest_float('weight_decay', 0.000001, 0.001, log=True)

    num_epochs = 400

    # Create your model with the sampled hyperparameters
    model = CATTransformer(alpha = alpha,
                           embed_size= embed_size,
                           n_cont = len(cont_columns),
                           cat_feat=cat_columns,
                           num_layers=num_layers,
                           heads=heads,
                           forward_expansion=forward_expansion,
                           decoder_dropout=decoder_dropout,
                           classification_dropout=classification_dropout,
                           pre_norm_on=pre_norm_on,
                           mlp_scale_classification=mlp_scale_classification,
                           targets_classes=target_classes,
                           ).to(device_in_use)

    # Define loss function and optimizer
    loss_function = nn.CrossEntropyLoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate, weight_decay=weight_decay)

    # Initialize early stopping
    early_stopping = EarlyStopping(patience=10)  # Adjust patience as needed

    # Training loop with a large number of epochs
    for epoch in range(num_epochs):
        train_loss, train_acc, train_f1= train(regression_on=False, 
                                  get_attn=False,
                                   dataloader=train_dataloader, 
                                   model=model, 
                                   loss_function=loss_function, 
                                   optimizer=optimizer, 
                                   device_in_use=device_in_use)
        val_loss, val_acc, val_f1 = test(regression_on=False,
                               get_attn=False,
                               dataloader=val_dataloader,
                               model=model,
                               loss_function=loss_function,
                               device_in_use=device_in_use)
        # Check if we should early stop based on validation accuracy
        if early_stopping(val_acc):
            break

    
    # Log the final test accuracy for this trial to a shared log file
    final_log = f"Trial {trial_number} completed. Validation Accuracy = {val_acc:.4f}"

    # Return the test accuracy as the objective to optimize
    return val_acc

In [4]:
# Set the number of optimization trials
num_trials = 100

# Create an Optuna study
study = optuna.create_study(direction='maximize')  # Maximize validation accuracy

# Start the optimization process
study.optimize(objective, n_trials=num_trials, show_progress_bar=True)

# Get the best hyperparameters and the validation accuracy at the point of early stopping
best_params = study.best_params
best_val_accuracy = study.best_value

print("Best Hyperparameters:", best_params)
print("Best Validation Accuracy (at Early Stopping):", best_val_accuracy)

[I 2024-01-06 16:20:51,819] A new study created in memory with name: no-name-0e35e79d-ca18-44d7-b5ae-2a45e9f4203b
Best trial: 0. Best value: 0.682736:   1%|          | 1/100 [55:27<91:30:09, 3327.37s/it]

[I 2024-01-06 17:16:19,193] Trial 0 finished with value: 0.6827361120554838 and parameters: {'sigma': 0.05734860207327919, 'num_layers': 2, 'embed_size': 100, 'heads': 5, 'forward_expansion': 1, 'prenorm_on': False, 'mlp_scale_classification': 8, 'decoder_dropout': 0.5, 'class_drop': 0.2, 'learning_rate': 1.2928913088829067e-05, 'weight_decay': 0.0002301621325063564}. Best is trial 0 with value: 0.6827361120554838.


Best trial: 0. Best value: 0.682736:   2%|▏         | 2/100 [1:57:26<96:51:18, 3557.95s/it]

[I 2024-01-06 18:18:18,542] Trial 1 finished with value: 0.6745767321683552 and parameters: {'sigma': 0.5211549566727209, 'num_layers': 2, 'embed_size': 160, 'heads': 5, 'forward_expansion': 2, 'prenorm_on': True, 'mlp_scale_classification': 7, 'decoder_dropout': 0.1, 'class_drop': 0.1, 'learning_rate': 0.0009952319555457472, 'weight_decay': 0.00012104854989983931}. Best is trial 0 with value: 0.6827361120554838.


Best trial: 0. Best value: 0.682736:   3%|▎         | 3/100 [3:06:36<103:08:55, 3828.20s/it]

...

[I 2024-01-11 02:38:35,151] Trial 97 finished with value: 0.7248249133065887 and parameters: {'sigma': 0.008266731664182285, 'num_layers': 1, 'embed_size': 60, 'heads': 10, 'forward_expansion': 1, 'prenorm_on': True, 'mlp_scale_classification': 8, 'decoder_dropout': 0.2, 'class_drop': 0, 'learning_rate': 6.66585426705999e-05, 'weight_decay': 3.872274137064446e-05}. Best is trial 8 with value: 0.7348881485007139.


Best trial: 8. Best value: 0.734888:  99%|█████████▉| 99/100 [107:03:21<46:06, 2766.72s/it]  

[I 2024-01-11 03:24:13,767] Trial 98 finished with value: 0.7296525464064731 and parameters: {'sigma': 0.012633702553838338, 'num_layers': 1, 'embed_size': 100, 'heads': 10, 'forward_expansion': 3, 'prenorm_on': True, 'mlp_scale_classification': 8, 'decoder_dropout': 0.2, 'class_drop': 0, 'learning_rate': 4.606435560401083e-05, 'weight_decay': 7.676447320745607e-05}. Best is trial 8 with value: 0.7348881485007139.


Best trial: 8. Best value: 0.734888: 100%|██████████| 100/100 [108:04:25<00:00, 3890.66s/it]

[I 2024-01-11 04:25:17,610] Trial 99 finished with value: 0.7261848099544435 and parameters: {'sigma': 0.009952717161968877, 'num_layers': 2, 'embed_size': 180, 'heads': 10, 'forward_expansion': 2, 'prenorm_on': True, 'mlp_scale_classification': 7, 'decoder_dropout': 0.2, 'class_drop': 0, 'learning_rate': 3.170065337440393e-05, 'weight_decay': 2.901399552417328e-05}. Best is trial 8 with value: 0.7348881485007139.
Best Hyperparameters: {'sigma': 0.02247218458508478, 'num_layers': 2, 'embed_size': 70, 'heads': 5, 'forward_expansion': 6, 'prenorm_on': True, 'mlp_scale_classification': 2, 'decoder_dropout': 0.1, 'class_drop': 0, 'learning_rate': 4.857253773800948e-05, 'weight_decay': 6.853700287640523e-05}
Best Validation Accuracy (at Early Stopping): 0.7348881485007139
